In [4]:
import requests, json

# register_url = "http://127.0.0.1:8000/quote/"
register_url = 'http://15.164.27.255/quote/'

def save(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def read(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def regiester(filename, url):
    data = read(filename)
    for quote in data[:10]:
        content = quote['content']
        response = requests.post(url, data=quote)
        if response.status_code == 201:
            print(f"전송 성공: {content}")
        else:
            print("전송 실패:", content, response.status_code, response.text)

### description 자동 생성

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = os.getenv("OPENAI_API_KEY") # OPENAI API KEY
)
def get_chatgpt_response(content):
    system_prompt = """
    너는 명언에 대해 설명을 달아주는 역할을 수행해야 해. 이 명언에 담긴 뜻과 함께 일상의 어떤 상황에서 해당 명언이 도움이 될 수 있을지를 고려해서 명언 설명을 작성해줘. 그리고 명언 구절을 재언급하지는 말아줘. 반드시 한국어 글자수 기준으로 300~400자 분량을 준수해야 해. '예를 들어', "~도 마찬가지다"와 같이 의미를 담지 않고 있는 연결 문장들은 다 제외해줘. 예제는 다음과 같아.
    명언 : "되찾을 수 없는게 세월이니 시시한 일에 시간을 낭비하지 말고 순간순간을 후회 없이 잘 살아야 한다",
    "description": "이 명언은 시간이 유한하고 되돌릴 수 없음을 상기시키며, 매 순간을 소중히 여기고 후회 없이 살아야 한다는 교훈을 줍니다. 시간은 누구에게나 공평하게 흐르며, 한 번 지나가면 다시 돌아오지 않습니다. 따라서 중요한 일에 집중하고, 의미 있는 삶을 살기 위해 노력하는 것이 중요합니다. 일상에서는 시간을 효율적으로 관리하고, 자신의 목표와 가치에 맞는 일에 에너지를 쏟는 데 도움이 됩니다. 예를 들어, 불필요한 걱정이나 갈등에 시간을 소비하기보다는, 자신에게 진정으로 중요한 사람들과 시간을 보내고, 자신의 꿈과 목표를 향해 꾸준히 나아가는 것이 더 가치 있습니다. 또한, 일을 할 때도 단순한 반복 업무나 중요하지 않은 일에 지나치게 시간을 할애하지 않고, 우선순위를 정해 중요한 일을 먼저 처리하는 습관을 기르는 것이 필요합니다. 이 명언은 우리에게 매 순간을 소중히 여기고, 후회 없이 최선을 다해 살아가는 태도를 가지도록 격려합니다. 이를 통해 우리는 보다 의미 있고 충실한 삶을 살아갈 수 있습니다."
    """

    # 프롬프트 예제
    user_prompt = f"""
    "명언" : {content}
    "description" :
    """
    try:
        # ChatGPT API 호출
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,  # 응답의 창의성 조절 (0.0 ~ 1.0)
        )
        response_text = response.choices[0].message.content.strip()
        return response_text
    except Exception as e:
        return str(e)

# content, author만 있는 json 파일 로드
data = read('quote.json')

from tqdm import tqdm
new_data = []
for x in tqdm(data):
    temp = {}
    temp['content'] = x['content']
    temp['description'] = get_chatgpt_response(x['content'])
    temp['author'] = x['author']
    new_data.append(temp)

# 결과를 json파일로 저장
save(new_data, 'out.json')

### 서버 DB에 명언 자동 등록

In [ ]:
import requests, json
from tqdm import tqdm

def read(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data
quotes = read('quote_example.json') # 명언 100개 로드

access_token = [
    # user별 access token 리스트
] 
access_index = -1

# 상위 30개는 5개씩 6명의 유저 명목으로 생성
for idx, quote in tqdm(enumerate(quotes[:30])):
    if idx % 5 == 0: access_index += 1
    headers = {
        "Authorization": f"Bearer {access_token[access_index]}"
    }
    data = {
        "content" : quote['content'],
        "description" : quote['description']
    }
    response = requests.post("http://15.164.27.255/quote/quote-register/", headers=headers,data=data)
    if response.status_code != 201:
        print("에러 발생")
        break

# 남은 70개는 User Token 없이 삽입
for quote in quotes[30:]:
    content = quote['content']
    response = requests.post('http://15.164.27.255/quote/', data=quote)
    if response.status_code == 201:
        print(f"전송 성공: {content}")
    else:
        print("전송 실패:", content, response.status_code, response.text)

### Query로 질문에 대한 응답 예제파일 생성

In [33]:
import pandas as pd
def recommend_test(query_filename, url):
    query_set = pd.read_csv(query_filename) 
    result = []

    for query in query_set['text']:
        response = requests.post(url, data={'query' : query})
        if response.status_code == 200: # OK
            print("전송 성공")
            response_data = response.json()
            sorted_data = sorted(response_data, key=lambda x: x['score'])
            data = {
                "query" : query,
                "quote1" : sorted_data[0]['quote'],
                "description1" : sorted_data[0]['description'],
                "quote2" : sorted_data[1]['quote'],
                "description2" : sorted_data[1]['description'],
                "quote3" : sorted_data[2]['quote'],
                "description3" : sorted_data[2]['description'],
            }
            result.append(data)
        else:
            print("전송 실패:", response.status_code, response.text)
    return result

In [34]:
recommend_url = "http://127.0.0.1:8080/quote/recommend/"
query_filename = 'question.csv'

query_file = pd.read_csv(query_filename)
query_file.head()

,text
0,오늘 해야할 일이 있는 것을 아는데 너무 귀찮아서 그냥 집에서 누워서 핸드폰 보면서...
1,모든 것이 너무 힘들어. 요즘 너무 바빠서 처리할 일도 많고 그러다보니 여유도 없어...
2,오늘은 너무 행복한 날이야. 내가 1년 동안 준비했던 시험에서 원하는 성적을 거두었...
3,새로운 도전을 시작해볼까? 물론 지금 하는 일을 포기하고 도전해야 한다는 것이 두렵...
4,실수해서 너무 속상해. 많은 사람들 앞에서 창피를 당했어. 사람들이 내가 무능한 사...


In [35]:
result = recommend_test(query_filename, recommend_url)

전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공
전송 성공


In [32]:
with open('answer.json', 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)